<a href="https://colab.research.google.com/github/Dhruv1603/Natural_Language_Processing/blob/main/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Database link:
https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment

In [ ]:
# Import lib

import pandas as pd
from google.colab import drive
import spacy
import numpy as np
import pandas as pd
from google.colab import drive
import spacy
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier


nlp = spacy.load('en_core_web_sm')

In [ ]:
# Mount the drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
# Load the dataset
df = pd.read_csv('Tweets.csv', encoding='latin-1', header=None)

# Display the first few rows
print(df.head())


             0                  1  \
0     tweet_id  airline_sentiment   
1  5.70306E+17            neutral   
2  5.70301E+17           positive   
3  5.70301E+17            neutral   
4  5.70301E+17           negative   

                                                   2   3   4  
0                                               text NaN NaN  
1                @VirginAmerica What @dhepburn said. NaN NaN  
2  @VirginAmerica plus you've added commercials t... NaN NaN  
3  @VirginAmerica I didn't today... Must mean I n... NaN NaN  
4  @VirginAmerica it's really aggressive to blast... NaN NaN  


In [ ]:
df.isnull().sum()

,0
0,0
1,0
2,0
3,14641
4,14641


In [ ]:
df_sample = df.sample(frac=0.1)

print(df_sample)


                 0         1  \
5757   5.68659E+17   neutral   
523    5.70298E+17   neutral   
2928   5.68828E+17  negative   
6303   5.68072E+17   neutral   
1352   5.69785E+17   neutral   
...            ...       ...   
12430  5.70192E+17  negative   
7127   5.69928E+17  negative   
10281  5.69399E+17  negative   
10710  5.68959E+17  negative   
2870   5.68855E+17   neutral   

                                                       2   3   4  
5757   @SouthwestAir would love to win tickets and ta... NaN NaN  
523    @united no I don't, but I'm sure United have m... NaN NaN  
2928   @united trying to reach him for the number. At... NaN NaN  
6303          @SouthwestAir Ahhhh! Sorry, just followed. NaN NaN  
1352                   @united followed you and DM'd you NaN NaN  
...                                                  ...  ..  ..  
12430  @AmericanAir she has  the guy u served drinks ... NaN NaN  
7127   â@JetBlue: Our fleet's on fleek. http://t.co... NaN NaN  
10281  @USAi

In [ ]:
# Preprocess all data

def preprocess_text(text):
  # Remove URLs
  text = re.sub(r'http\S+', '', text)
  # Remove mentions
  text = re.sub(r'@\S+', '', text)
  # Remove hashtags
  text = re.sub(r'#\S+', '', text)
  # Remove special characters and punctuation
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  # Convert to lowercase
  text = text.lower()
  # Remove extra whitespace
  text = ' '.join(text.split())
  return text

# Apply the preprocessing function to the 'text' column
df_sample['processed_text'] = df_sample[2].apply(preprocess_text)
df_sample['processed_sentiments'] = df_sample[1].apply(preprocess_text)

print(df_sample['processed_sentiments'], df_sample['processed_text'])


5757      neutral
523       neutral
2928     negative
6303      neutral
1352      neutral
           ...   
12430    negative
7127     negative
10281    negative
10710    negative
2870      neutral
Name: processed_sentiments, Length: 1464, dtype: object 5757     would love to win tickets and take my son to i...
523      no i dont but im sure united have my info on i...
2928     trying to reach him for the number at last the...
6303                             ahhhh sorry just followed
1352                              followed you and dmd you
                               ...                        
12430    she has the guy u served drinks 2 til he neede...
7127                      our fleets on fleek make it stop
10281    this is ridiculous i have a 600 am flight and ...
10710                  already tried how about conf via dm
2870     yes ive boarded this way many times amp have n...
Name: processed_text, Length: 1464, dtype: object


In [ ]:

# Split the data into train, validation, and test sets
train_data, temp_data = train_test_split(df_sample, test_size=0.3, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print("Train data shape:", train_data.shape)
print("Validation data shape:", valid_data.shape)
print("Test data shape:", test_data.shape)


Train data shape: (1024, 7)
Validation data shape: (220, 7)
Test data shape: (220, 7)


In [ ]:
# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
train_bow = vectorizer.fit_transform(train_data['processed_text'])

# Transform validation and test data using the fitted vectorizer
valid_bow = vectorizer.transform(valid_data['processed_text'])
test_bow = vectorizer.transform(test_data['processed_text'])

print("Train BoW shape:", train_bow.shape)
print("Validation BoW shape:", valid_bow.shape)
print("Test BoW shape:", test_bow.shape)


Train BoW shape: (1024, 2823)
Validation BoW shape: (220, 2823)
Test BoW shape: (220, 2823)


## Model 01

In [ ]:
# Initialize Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(train_bow, train_data['processed_sentiments'])


LogisticRegression()

In [ ]:
# Train the model
model.fit(train_bow, train_data['processed_sentiments'])

# Evaluate the model on the validation set
valid_accuracy = model.score(valid_bow, valid_data['processed_sentiments'])
print("Validation Accuracy:", valid_accuracy)


Validation Accuracy: 0.7636363636363637


In [ ]:
# Evaluate the model on the test set
test_accuracy = model.score(test_bow, test_data['processed_sentiments'])
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.740909090909091


In [ ]:
# Experiment with different autoencoder architectures
hidden_layer_sizes = [(100,), (50, 50), (100, 50, 100)]
activations = ['relu', 'tanh']

for hidden_layer_size in hidden_layer_sizes:
  for activation in activations:
    # Initialize autoencoder
    autoencoder = MLPRegressor(hidden_layer_sizes=hidden_layer_size, activation=activation)

    # Train the autoencoder
    autoencoder.fit(train_bow.toarray(), train_bow.toarray())

    # Reconstruct the training data
    train_bow_reconstructed = autoencoder.predict(train_bow.toarray()) # Convert train_bow to dense array

    # Calculate reconstruction error
    train_reconstruction_error = np.mean(np.square(train_bow.toarray() - train_bow_reconstructed), axis=1) # Convert train_bow to dense array

    # Set a threshold for anomaly detection
    threshold = np.percentile(train_reconstruction_error, 95) # Example threshold

    # Identify anomalies in the training data
    train_anomalies = train_reconstruction_error > threshold

    # Remove anomalies from the training data
    train_bow_filtered = train_bow[~train_anomalies]
    train_sentiments_filtered = train_data['processed_sentiments'][~train_anomalies]

    # Train the logistic regression model on the filtered data
    model = LogisticRegression()
    model.fit(train_bow_filtered, train_sentiments_filtered)

    # Evaluate the model on the validation set
    valid_bow_reconstructed = autoencoder.predict(valid_bow.toarray()) # Convert valid_bow to dense array
    valid_reconstruction_error = np.mean(np.square(valid_bow.toarray() - valid_bow_reconstructed), axis=1) # Convert valid_bow to dense array
    valid_anomalies = valid_reconstruction_error > threshold
    valid_bow_filtered = valid_bow[~valid_anomalies]
    valid_sentiments_filtered = valid_data['processed_sentiments'][~valid_anomalies]
    valid_accuracy = model.score(valid_bow_filtered, valid_sentiments_filtered)
    print("Validation Accuracy (hidden layers: {}, activation: {}): {}".format(hidden_layer_size, activation, valid_accuracy))

# Further fine-tune hyperparameters and thresholds based on validation results

Validation Accuracy (hidden layers: (100,), activation: relu): 0.7277227722772277
Validation Accuracy (hidden layers: (100,), activation: tanh): 0.7348066298342542
Validation Accuracy (hidden layers: (50, 50), activation: relu): 0.7560975609756098
Validation Accuracy (hidden layers: (50, 50), activation: tanh): 0.7330097087378641
Validation Accuracy (hidden layers: (100, 50, 100), activation: relu): 0.7378640776699029
Validation Accuracy (hidden layers: (100, 50, 100), activation: tanh): 0.7352941176470589


In [ ]:
# Improve the model's generalization and robustness

import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression

# Experiment with different autoencoder architectures and regularization
hidden_layer_sizes = [(100,), (50, 50), (100, 50, 100)]
activations = ['relu', 'tanh']
alphas = [0.001, 0.01, 0.1]

best_valid_accuracy = 0
best_params = {}

for hidden_layer_size in hidden_layer_sizes:
  for activation in activations:
    for alpha in alphas:
      # Initialize autoencoder with L2 regularization
      autoencoder = MLPRegressor(hidden_layer_sizes=hidden_layer_size,
                                activation=activation,
                                alpha=alpha)

      # Train the autoencoder
      autoencoder.fit(train_bow.toarray(), train_bow.toarray())

      # Reconstruct the training data
      train_bow_reconstructed = autoencoder.predict(train_bow.toarray())

      # Calculate reconstruction error
      train_reconstruction_error = np.mean(np.square(train_bow.toarray() - train_bow_reconstructed), axis=1)

      # Set a threshold for anomaly detection
      threshold = np.percentile(train_reconstruction_error, 95)

      # Identify anomalies in the training data
      train_anomalies = train_reconstruction_error > threshold

      # Remove anomalies from the training data
      train_bow_filtered = train_bow[~train_anomalies]
      train_sentiments_filtered = train_data['processed_sentiments'][~train_anomalies]

      # Train the logistic regression model on the filtered data with L2 regularization
      model = LogisticRegression(penalty='l2', C=1/alpha)
      model.fit(train_bow_filtered, train_sentiments_filtered)

      # Evaluate the model on the validation set
      valid_bow_reconstructed = autoencoder.predict(valid_bow.toarray())
      valid_reconstruction_error = np.mean(np.square(valid_bow.toarray() - valid_bow_reconstructed), axis=1)
      valid_anomalies = valid_reconstruction_error > threshold
      valid_bow_filtered = valid_bow[~valid_anomalies]
      valid_sentiments_filtered = valid_data['processed_sentiments'][~valid_anomalies]
      valid_accuracy = model.score(valid_bow_filtered, valid_sentiments_filtered)

      print("Validation Accuracy (hidden layers: {}, activation: {}, alpha: {}): {}".format(hidden_layer_size, activation, alpha, valid_accuracy))

      # Update best parameters if validation accuracy improves
      if valid_accuracy > best_valid_accuracy:
        best_valid_accuracy = valid_accuracy
        best_params = {'hidden_layer_size': hidden_layer_size,
                       'activation': activation,
                       'alpha': alpha}

print("Best Validation Accuracy:", best_valid_accuracy)
print("Best Parameters:", best_params)

# Train the final model using the best parameters
final_autoencoder = MLPRegressor(hidden_layer_sizes=best_params['hidden_layer_size'],
                              activation=best_params['activation'],
                              alpha=best_params['alpha'])
final_autoencoder.fit(train_bow.toarray(), train_bow.toarray())


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy (hidden layers: (100,), activation: relu, alpha: 0.001): 0.6836734693877551


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy (hidden layers: (100,), activation: relu, alpha: 0.01): 0.686046511627907
Validation Accuracy (hidden layers: (100,), activation: relu, alpha: 0.1): 0.7337278106508875


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy (hidden layers: (100, 50, 100), activation: tanh, alpha: 0.001): 0.6716417910447762


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy (hidden layers: (100, 50, 100), activation: tanh, alpha: 0.01): 0.7192118226600985
Validation Accuracy (hidden layers: (100, 50, 100), activation: tanh, alpha: 0.1): 0.7525252525252525
Best Validation Accuracy: 0.7562189054726368
Best Parameters: {'hidden_layer_size': (50, 50), 'activation': 'tanh', 'alpha': 0.1}


MLPRegressor(activation='tanh', alpha=0.1, hidden_layer_sizes=(50, 50))

## Model 02

In [ ]:
# Random Forest model

# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(train_bow, train_data['processed_sentiments'])


RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate the model on the validation set
valid_accuracy = rf_model.score(valid_bow, valid_data['processed_sentiments'])
print("Random Forest Validation Accuracy:", valid_accuracy)


Random Forest Validation Accuracy: 0.7090909090909091


In [ ]:

# Evaluate the model on the test set
test_accuracy = rf_model.score(test_bow, test_data['processed_sentiments'])
print("Random Forest Test Accuracy:", test_accuracy)

Random Forest Test Accuracy: 0.6818181818181818


In [ ]:
# Fine-tune the model's performance

# Experiment with different autoencoder architectures
hidden_layer_sizes = [(100,), (50, 50), (100, 50, 100)]
activations = ['relu', 'tanh']
learning_rates = [0.01, 0.001]

best_valid_accuracy = 0
best_params = {}

for hidden_layer_size in hidden_layer_sizes:
  for activation in activations:
    for learning_rate in learning_rates:
      # Initialize autoencoder
      autoencoder = MLPRegressor(hidden_layer_sizes=hidden_layer_size,
                                 activation=activation,
                                 learning_rate_init=learning_rate)

      # Train the autoencoder
      autoencoder.fit(train_bow.toarray(), train_bow.toarray())

      # Reconstruct the training data
      train_bow_reconstructed = autoencoder.predict(train_bow.toarray())

      # Calculate reconstruction error
      train_reconstruction_error = np.mean(np.square(train_bow.toarray() - train_bow_reconstructed), axis=1)

      # Experiment with different thresholds
      for threshold_percentile in range(90, 100):
        threshold = np.percentile(train_reconstruction_error, threshold_percentile)

        # Identify anomalies in the training data
        train_anomalies = train_reconstruction_error > threshold

        # Remove anomalies from the training data
        train_bow_filtered = train_bow[~train_anomalies]
        train_sentiments_filtered = train_data['processed_sentiments'][~train_anomalies]

        # Train the logistic regression model on the filtered data
        model = LogisticRegression()
        model.fit(train_bow_filtered, train_sentiments_filtered)

        # Evaluate the model on the validation set
        valid_bow_reconstructed = autoencoder.predict(valid_bow.toarray())
        valid_reconstruction_error = np.mean(np.square(valid_bow.toarray() - valid_bow_reconstructed), axis=1)
        valid_anomalies = valid_reconstruction_error > threshold
        valid_bow_filtered = valid_bow[~valid_anomalies]
        valid_sentiments_filtered = valid_data['processed_sentiments'][~valid_anomalies]
        valid_accuracy = model.score(valid_bow_filtered, valid_sentiments_filtered)

        # Update best parameters if validation accuracy improves
        if valid_accuracy > best_valid_accuracy:
          best_valid_accuracy = valid_accuracy
          best_params = {
              'hidden_layer_size': hidden_layer_size,
              'activation': activation,
              'learning_rate': learning_rate,
              'threshold_percentile': threshold_percentile
          }

print("Best Validation Accuracy:", best_valid_accuracy)
print("Best Parameters:", best_params)

Best Validation Accuracy: 0.7669902912621359
Best Parameters: {'hidden_layer_size': (50, 50), 'activation': 'tanh', 'learning_rate': 0.01, 'threshold_percentile': 99}


In [ ]:
# Improve the model's generalization and robustness

# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(train_bow, train_data['processed_sentiments'])

# Evaluate the model on the validation set
valid_accuracy = rf_model.score(valid_bow, valid_data['processed_sentiments'])
print("Random Forest Validation Accuracy:", valid_accuracy)


# Evaluate the model on the test set
test_accuracy = rf_model.score(test_bow, test_data['processed_sentiments'])
print("Random Forest Test Accuracy:", test_accuracy)

# Experiment with different autoencoder architectures and improved techniques
hidden_layer_sizes = [(100,), (50, 50), (100, 50, 100)]
activations = ['relu', 'tanh']
learning_rates = [0.01, 0.001]
dropout_rates = [0.2, 0.5]
l2_regs = [0.001, 0.01]

best_valid_accuracy = 0
best_params = {}

for hidden_layer_size in hidden_layer_sizes:
  for activation in activations:
    for learning_rate in learning_rates:
      for dropout_rate in dropout_rates:
        for l2_reg in l2_regs:
          # Initialize autoencoder with L2 regularization and dropout
          autoencoder = Sequential()
          for units in hidden_layer_size:
            autoencoder.add(Dense(units, activation=activation, kernel_regularizer='l2'))
            autoencoder.add(Dropout(dropout_rate))
          autoencoder.add(Dense(train_bow.shape[1]))

          optimizer = Adam(learning_rate=learning_rate)
          autoencoder.compile(optimizer=optimizer, loss='mse')

          # Learning rate scheduling
          lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

          # Train the autoencoder
          autoencoder.fit(train_bow.toarray(), train_bow.toarray(),
                          epochs=5, batch_size=32,
                          validation_data=(valid_bow.toarray(), valid_bow.toarray()),
                          callbacks=[lr_scheduler])

          # Reconstruct the training data
          train_bow_reconstructed = autoencoder.predict(train_bow.toarray())

          # Calculate reconstruction error
          train_reconstruction_error = np.mean(np.square(train_bow.toarray() - train_bow_reconstructed), axis=1)

          # Experiment with different thresholds
          for threshold_percentile in range(90, 100):
            threshold = np.percentile(train_reconstruction_error, threshold_percentile)

            # Identify anomalies in the training data
            train_anomalies = train_reconstruction_error > threshold

            # Remove anomalies from the training data
            train_bow_filtered = train_bow[~train_anomalies]
            train_sentiments_filtered = train_data['processed_sentiments'][~train_anomalies]

            # Train the logistic regression model on the filtered data
            model = LogisticRegression()
            model.fit(train_bow_filtered, train_sentiments_filtered)

            # Evaluate the model on the validation set
            valid_bow_reconstructed = autoencoder.predict(valid_bow.toarray())
            valid_reconstruction_error = np.mean(np.square(valid_bow.toarray() - valid_bow_reconstructed), axis=1)
            valid_anomalies = valid_reconstruction_error > threshold
            valid_bow_filtered = valid_bow[~valid_anomalies]
            valid_sentiments_filtered = valid_data['processed_sentiments'][~valid_anomalies]
            valid_accuracy = model.score(valid_bow_filtered, valid_sentiments_filtered)

            # Update best parameters if validation accuracy improves
            if valid_accuracy > best_valid_accuracy:
              best_valid_accuracy = valid_accuracy
              best_params = {
                  'hidden_layer_size': hidden_layer_size,
                  'activation': activation,
                  'learning_rate': learning_rate,
                  'dropout_rate': dropout_rate,
                  'l2_reg': l2_reg,
                  'threshold_percentile': threshold_percentile
              }

print("Best Validation Accuracy:", best_valid_accuracy)
print("Best Parameters:", best_params)


Random Forest Validation Accuracy: 0.7090909090909091
Random Forest Test Accuracy: 0.6818181818181818
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.4821 - val_loss: 0.0288 - learning_rate: 0.0100
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0170 - val_loss: 0.0060 - learning_rate: 0.0100
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0059 - val_loss: 0.0052 - learning_rate: 0.0100
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0056 - val_loss: 0.0052 - learning_rate: 0.0100
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0056 - val_loss: 0.0052 - learning_rate: 0.0100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━

In [ ]:
# Best model
print("Best Validation Accuracy:", best_valid_accuracy)
print("Best Parameters:", best_params)

Best Validation Accuracy: 0.7592592592592593
Best Parameters: {'hidden_layer_size': (100,), 'activation': 'relu', 'learning_rate': 0.01, 'dropout_rate': 0.5, 'l2_reg': 0.001, 'threshold_percentile': 98}


In [ ]:
# Reference: https://www.kaggle.com/code/ayu1391994/nlp-using-random-forest
# Reference: https://medium.com/analytics-vidhya/randomforest-classifier-vs-multinomial-naive-bayes-for-a-multi-output-natural-language-2426381a5217
# Reference: https://towardsdatascience.com/nlp-101-%E2%85%94-linear-models-for-text-classification-8ced8199c2a8
# Reference: https://lajavaness.medium.com/regression-with-text-input-using-bert-and-transformers-71c155034b13
# Reference: https://medium.com/@maleeshadesilva21/preprocessing-steps-for-natural-language-processing-nlp-a-beginners-guide-d6d9bf7689c9